In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [88]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import pickle
import seaborn as sns

In [89]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [90]:
df = df.sample(10000)

In [91]:
df.shape

(10000, 2)

In [92]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [93]:
df["sentiment"].unique()

array(['negative', 'positive'], dtype=object)

In [94]:
df["sentiment"].value_counts()

sentiment
negative    5018
positive    4982
Name: count, dtype: int64

In [95]:
label = LabelEncoder()
df["sentiment"] = label.fit_transform(df["sentiment"])

In [96]:
# x = df["review"]
# y = df["sentiment"]

In [97]:
x

25321    This is, without doubt, one of the worst films...
26013    This is a great little movie, full of interest...
25500    To say that this is a good show is not to say ...
4013     Quick summary of the book: Boy, Billy Tepper, ...
7157     I can't believe I actually spent almost three ...
                               ...                        
48281    Some might remember if having seen the film Ju...
2842     Seriously, it had everything you could want in...
17379    Using footage pillaged from Planet of Dinosaur...
11842    If Ashanti had been a serious attempt at a fil...
33958    In my personal opinion i think this is the gre...
Name: review, Length: 10000, dtype: object

In [98]:
def process(text):
    y = []
    for i in text.split():
        y.append(re.sub("[^a-zA-Z]","",i))
    return " ".join(y) 

In [99]:
df["review"] = df["review"].apply(process)

In [100]:
df["review"] = df["review"].apply(lambda x:x.lower())

In [101]:
ps = PorterStemmer()
def Stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return" ".join(y)

In [102]:
df["review"] = df["review"].apply(Stem)

In [123]:
df

,review,sentiment
25321,thi is without doubt one of the worst film ive...,0
26013,thi is a great littl movi full of interest cha...,1
25500,to say that thi is a good show is not to say a...,1
4013,quick summari of the book boy billi tepper abo...,1
7157,i cant believ i actual spent almost three hour...,0
...,...,...
48281,some might rememb if have seen the film juno a...,0
2842,serious it had everyth you could want in a mov...,1
17379,use footag pillag from planet of dinosaur thi ...,0
11842,if ashanti had been a seriou attempt at a film...,0


In [135]:
x = df["review"]
y = df["sentiment"]

In [125]:
cv = CountVectorizer(max_features = 5000,stop_words = "english")
vectors = cv.fit_transform(df["review"]).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [137]:
x_train,x_test,y_train,y_test = train_test_split(vectors,y,test_size=0.2,random_state = 42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((8000, 5000), (2000, 5000), (8000,), (2000,))

In [138]:
clf = MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB()

In [139]:
predict = clf.predict(x_test)

In [140]:
print(accuracy_score(y_test,predict))
print(classification_report(y_test,predict))

0.8245
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1005
           1       0.83      0.81      0.82       995

    accuracy                           0.82      2000
   macro avg       0.82      0.82      0.82      2000
weighted avg       0.82      0.82      0.82      2000



In [141]:
pickle.dump(cv,open("count-vectoriser.pkl","wb"))
pickle.dump(clf,open("movie_review_classification.pkl","wb"))